<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [13]:
import pandas as pd
import numpy as np
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagementOpen/main/financial_models/Opportunities/Screener/screener_summary_latest.csv'
df = pd.read_csv(summary_url)
# Preparing the data
df['Last_fy'] = pd.to_datetime(df['Last_fy'])
# Missing Forex_rate
mop_hkd = (df["Price_currency"] == "HKD") & (df["Reporting_Currency"] == "MOP")
twd_hkd = (df["Price_currency"] == "HKD") & (df["Reporting_Currency"] == "TWD")
twd_usd = (df["Price_currency"] == "USD") & (df["Reporting_Currency"] == "TWD")
df.loc[mop_hkd, "Fx_rate"] = 0.98
df.loc[mop_hkd, "Fx_rate"] = 0.26
df.loc[twd_usd, "Fx_rate"] = 7.81
# exclude minor countries
df = df[df['Fx_rate'].notna()]
df = df.fillna(0)

In [ ]:
"""0. Definitions"""
# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price / df['Fx_rate']
# dividend rate & buyback rate
dividend_rate = df['Dividend'] / df['Price']
buyback_rate = df['Buyback'] / df['Price']
# Capital Structure related
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
common_equity = df['TotalEquityGrossMinorityInterest'] - df['MinorityInterest']
net_working_capital = np.where((df['CurrentAssets'] == 0) & (df['CurrentLiabilities'] == 0), common_equity, df['CurrentAssets'] - df['CurrentLiabilities'])
excess_cash = np.where(net_working_capital >= df['CashAndCashEquivalents'], df['CashAndCashEquivalents'], net_working_capital)
# Operating assets and liabilities
op_assets = df['TotalAssets'] - df['CashAndCashEquivalents']
op_liabilities = df['TotalAssets'] - df['TotalEquityGrossMinorityInterest'] - total_debt
net_op_assets = op_assets - op_liabilities
# Non-operating assets
current_financial_assets = df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
nonop_noncash_assets = 0.75 * (df['InvestmentProperties'] + df['LongTermEquityInvestment']) + current_financial_assets

"""1. Stability Ratios"""
# liquidity_coverage_ratio
core_lcr = df['CashAndCashEquivalents'] / df['CurrentLiabilities']
lcr = (df['CashAndCashEquivalents'] + current_financial_assets) / df['CurrentLiabilities']
# leverage ratio
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
debt_ratio = total_debt / common_equity
# Accrual anomaly - Scaled Total Accruals
# sta = (df['NetIncomeCommonStockholders'] - df['CFO']) / df['TotalAssets']
# Scaled Net Operating Assets
sona = net_op_assets / df['TotalAssets']
# Todo: Dr. Messod Beneish's PROBM model

"""2. Quality Ratios: """
ppe_multiple = df['NetPPE'] / df['TotalRevenue']
greenblatt_capital = df['NetPPE'] + net_working_capital - df['CashAndCashEquivalents']  # Not consider capital structure
greenblatt_roc = df['EBIT'] / greenblatt_capital

"""3. Price Ratios: """
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - excess_cash - nonop_noncash_assets
ebit_tev = df['EBIT'] / enterprise_value

"""4. Momentum Ratios"""
# Todo: F-score

# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [15]:
# common fitlering conditions: 
# 1. Reasonable leverage and Good Liqudity
common_1 = (core_lcr >= 0.6) & (lcr >= 0.8)
common_2 = (current_ratio >= 1) & (debt_ratio <= 0.8)
common_3 = (total_debt / df['CurrentAssets']) < 1.5
# 2. Avoid negative gross margin and value trap
common_4 = capitalization_price > 1000000000
common_5 = (capitalization_price > 6000000000) & (df['Avg_Gross_margin'] > 0.01)
common_6 = (capitalization_price <= 6000000000) & ((dividend_rate > 0.01) | (buyback_rate > 0.01))

In [16]:
# 1st set of conditions: Stalwart
stalwart_1 = (df['Avg_Gross_margin'] > 10) & (df['Avg_ebit_margin'] > 15)
stalwart_2 = df['Avg_sales_growth'] >= 0.05
stalwart_3 = greenblatt_roc > 0.03

In [17]:
# 2nd set of conditions: Asset Play
asset_1 = (core_lcr >= 0.8) & (lcr >= 1)
asset_2 = (dividend_rate > 0.01) | (buyback_rate > 0.01)
asset_3 = (current_ratio >= 1.5) & (debt_ratio <= 0.4)

#Step 3. Output

Filter the dataset using the above conditions

In [18]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['EBIT/EV'] = ebit_tev
common_df['Dividend rate'] = dividend_rate
common_df['Buyback rate'] = buyback_rate
common_df['PPE_multiple'] = ppe_multiple
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4 & (common_5 | common_6)]
# display(common_df)

In [ ]:
# fitlered by 1st set of conditions: Stalwart
Stalwart_df = common_df
Stalwart_df = Stalwart_df.loc[stalwart_1 & stalwart_2 & stalwart_3]
print(Stalwart_df.to_string())

In [ ]:
# filtered by 2nd set of conditions: Asset Play
asset_df = common_df
asset_df = asset_df.loc[asset_1 & asset_2 & asset_3]
print(asset_df.to_string())

In [ ]:
# combine the results
result_set = pd.concat([Stalwart_df, asset_df])
result_set = result_set.groupby("Ticker").first()
print(result_set.to_string())

In [25]:
# Listing Location
market = 'HK' #@param ["HK","CN", "US", "JP"]

# HK only
if market == 'HK':
  exchange_condition = result_set['Exchange'] == 'HKG'
elif market == 'CN':
# A shares only
  exchange_condition = (result_set['Exchange'] == 'SHZ') | (result_set['Exchange'] == 'SHH')
elif market == 'US':
  exchange_condition = (result_set['Exchange'] == 'NMS') | (result_set['Exchange'] == 'NYQ')
else:
  exchange_condition = (result_set['Exchange'] == 'JPX')
display_set = result_set.loc[exchange_condition].sort_values(by=['EBIT/EV'], ascending=[True]).reset_index()
print(display_set.to_string())


      Ticker             Name Exchange    Price Price_currency       Shares Reporting_Currency   Fx_rate  Dividend    Buyback                    Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV    EBIT/EV  Dividend rate  Buyback rate  PPE_multiple
0    9982.HK  CENTRALCHINA MT      HKG    0.630            HKD   3282660096                CNY  1.151104  0.071836 